In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.rnn import BasicLSTMCell
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def testcase_shuffle_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]

    all_df = pd.concat([train_df[['clean_text', 'class']],
                        test_df[['clean_text', 'class']],
                        added_df[['clean_text', 'class']]])

    all_df = all_df.sample(len(all_df), random_state=10)
    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df.iloc[1062:]
    test_df = all_df.iloc[:1062]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [6]:
def testcase_add_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    added_df = pd.read_csv(
        '/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/added_dataset.csv',
        sep='\t')
    added_df['clean_text'] = added_df['sent'].apply(lambda x:x.lower())

    # class 제거
    added_df = added_df[added_df['class'].apply(lambda x: True if x in cols else False)]
    added_df.info()

    # duplicated sent 제거
    added_df = added_df[added_df['clean_text'].apply(lambda x: False if x in df['clean_text'].tolist() else True)]
    added_df.info()

    # train_df merge

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size * (1 - rate))]
    test_df = df.iloc[int(size * (1 - rate)):]
    train_df['setlabel'] = 1
    test_df['setlabel'] = 0
    added_df['setlabel'] = 2

    all_df = pd.concat([train_df[['clean_text', 'class', 'setlabel']],
                        test_df[['clean_text', 'class', 'setlabel']],
                        added_df[['clean_text', 'class','setlabel']]])

    all_df.reset_index(drop=True, inplace=True)

    label_df = pd.get_dummies(all_df['class'])

    train_df = all_df[all_df['setlabel'] != 0]
    test_df = all_df[all_df['setlabel'] == 0]

    cols = label_df.columns

    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[
            test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [7]:
def digi_data_load():
    df = pd.read_csv('/Users/1003874/bsdev/Text-classification-with-CNN-RNN-with-Tensorflow/Ch01_Data_load/data/preprocessed_mart_digi_speech_act.csv')
    df['clean_text'] = df['clean_text'].apply(lambda x:x.lower())
    s = df['class'].value_counts()
    cols = s[s>50].index.tolist()

    sub_list = []
    for col in cols:
        sub_list.append(df[df['class'] == col])
    df = pd.concat(sub_list, axis=0)
    df = df.sample(len(df), random_state=10).reset_index(drop=True)

    size = len(df)
    rate = 0.2
    train_df = df.iloc[:int(size*(1-rate))]
    test_df = df.iloc[int(size*(1-rate)):]
    label_df = pd.get_dummies(df['class'])
    cols = label_df.columns
    print(len(train_df), len(test_df))

    TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL = \
        train_df['clean_text'].values, label_df.loc[train_df.index].values, test_df['clean_text'].values, label_df.loc[test_df.index].values

    return TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols

In [8]:
class ABLSTM(object):
    def __init__(self, config):
        self.max_len = config["max_len"]
        self.hidden_size = config["hidden_size"]
        self.vocab_size = config["vocab_size"]
        self.embedding_size = config["embedding_size"]
        self.n_class = config["n_class"]
        self.learning_rate = config["learning_rate"]

        # placeholder
        self.x = tf.placeholder(tf.int32, [None, self.max_len])
        self.label = tf.placeholder(tf.int32, [None])
        self.keep_prob = tf.placeholder(tf.float32)

    def build_graph(self):
        print("building graph")
        # Word embedding
        embeddings_var = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0),
                                     trainable=True)
        batch_embedded = tf.nn.embedding_lookup(embeddings_var, self.x)

        rnn_outputs, _ = bi_rnn(BasicLSTMCell(self.hidden_size),
                                BasicLSTMCell(self.hidden_size),
                                inputs=batch_embedded, dtype=tf.float32)

        fw_outputs, bw_outputs = rnn_outputs

        W = tf.Variable(tf.random_normal([self.hidden_size], stddev=0.1))
        H = fw_outputs + bw_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
        M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

        self.alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, self.hidden_size]),
                                                        tf.reshape(W, [-1, 1])),
                                              (-1, self.max_len)))  # batch_size x seq_len
        r = tf.matmul(tf.transpose(H, [0, 2, 1]),
                      tf.reshape(self.alpha, [-1, self.max_len, 1]))
        r = tf.squeeze(r)
        h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

        h_drop = tf.nn.dropout(h_star, self.keep_prob)

        # Fully connected layer（dense layer)
        FC_W = tf.Variable(tf.truncated_normal([self.hidden_size, self.n_class], stddev=0.1))
        FC_b = tf.Variable(tf.constant(0., shape=[self.n_class]))
        y_hat = tf.nn.xw_plus_b(h_drop, FC_W, FC_b)

        self.loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=self.label))

        # prediction
        self.prediction = tf.argmax(tf.nn.softmax(y_hat), 1)

        # optimization
        loss_to_minimize = self.loss
        tvars = tf.trainable_variables()
        gradients = tf.gradients(loss_to_minimize, tvars, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
        grads, global_norm = tf.clip_by_global_norm(gradients, 1.0)

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.train_op = self.optimizer.apply_gradients(zip(grads, tvars), global_step=self.global_step,
                                                       name='train_step')
        print("graph built successfully!")


In [9]:
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = digi_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_add_data_load()
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_shuffle_data_load()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29769 entries, 1 to 38319
Data columns (total 4 columns):
sent          29769 non-null object
nlu_result    29769 non-null object
class         29769 non-null object
clean_text    29769 non-null object
dtypes: object(4)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28516 entries, 1 to 38319
Data columns (total 4 columns):
sent          28516 non-null object
nlu_result    28516 non-null object
class         28516 non-null object
clean_text    28516 non-null object
dtypes: object(4)
memory usage: 1.1+ MB
32760 1062


In [10]:
train_intent = np.apply_along_axis(np.argmax,1,TRAIN_LABEL)
test_intent = np.apply_along_axis(np.argmax,1,TEST_LABEL)

In [11]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))

CPU times: user 51.3 s, sys: 736 ms, total: 52 s
Wall time: 40.1 s


In [12]:
y_train = pd.Series(train_intent)
y_test = pd.Series(test_intent)

In [13]:
x_train, x_test, vocab_size = \
        data_preprocessing_v2(train_doc, test_doc, max_len=32)

In [14]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 32,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 33,
    "learning_rate": 1e-3,
    "batch_size": 4,
    "train_epoch": 10
}

classifier = ABLSTM(config)
classifier.build_graph()

Validation Size:  1062
building graph
graph built successfully!


In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
dev_batch = (x_dev, y_dev)
start = time.time()
for e in range(config["train_epoch"]):

    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch))
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
    t1 = time.time()

    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))


Epoch 1 start !
Train Epoch time:  440.280 s
validation accuracy: 0.782 
Epoch 2 start !
Train Epoch time:  444.258 s
validation accuracy: 0.814 
Epoch 3 start !
Train Epoch time:  440.130 s
validation accuracy: 0.827 
Epoch 4 start !
Train Epoch time:  395.329 s
validation accuracy: 0.846 
Epoch 5 start !
Train Epoch time:  396.364 s
validation accuracy: 0.841 
Epoch 6 start !
Train Epoch time:  412.539 s
validation accuracy: 0.838 
Epoch 7 start !
Train Epoch time:  450.211 s
validation accuracy: 0.839 
Epoch 8 start !
Train Epoch time:  455.637 s
validation accuracy: 0.830 
Epoch 9 start !
Train Epoch time:  454.961 s
validation accuracy: 0.842 
Epoch 10 start !
Train Epoch time:  458.192 s
validation accuracy: 0.839 
Training finished, time consumed :  4350.051157951355  s
Start evaluating:  

Test accuracy : 83.867925 %


In [16]:
preds = []
reals = []
attns = []
for x_batch, y_batch in fill_feed_dict(x_test, y_test,1062, isshuffle=False):
    pred = get_prediction(classifier, sess, (x_batch, y_batch))
    attn = get_attn_weight(classifier, sess, (x_batch, y_batch))
    preds += pred.tolist()
    attns += attn.tolist()
    reals += y_batch.tolist()

In [17]:
test_df = pd.DataFrame({'sent':test_doc,'attn':attns, 'pred':preds, 'real':reals})
test_df

,sent,attn,pred,real
0,다음 에 또 할께요,"[0.7413091063499451, 0.10890647023916245, 0.06...",5,5
1,그냥 이렇게 끝내면 안되죠 . 다른 해상도 를 추천 해 주어야죠,"[0.5718769431114197, 0.0025768401101231575, 0....",5,24
2,정기 결재 \ n 해지 \ n 했으니 \ n \ n 환불 바랍니다,"[0.0003935715649276972, 0.004506119526922703, ...",28,28
3,방송 할만 한 노트북,"[0.6014400720596313, 0.011593986302614212, 0.2...",24,7
4,번품 할려 구 요,"[0.9171026945114136, 0.07818534970283508, 0.00...",4,7
5,이용권 종료 하려구요 ~!,"[0.623607873916626, 0.36070579290390015, 0.014...",4,4
6,비밀 쿠폰 이 뭔 가요,"[0.00043983093928545713, 0.9958051443099976, 3...",25,25
7,삼 쥐 에어컨 찾아줘,"[0.010640528053045273, 0.001728722476400435, 0...",24,24
8,전화 도 안 오고,"[0.49906855821609497, 0.019158028066158295, 0....",17,17
9,비밀번호 어떻게 바꾸나요 ??,"[0.9460952877998352, 0.00432624714449048, 0.04...",4,29


In [18]:
test_df['real'] = test_df['real'].apply(lambda x: cols[x])
test_df['pred'] = test_df['pred'].apply(lambda x: cols[x])

In [19]:
f1_score(test_df['real'],test_df['pred'],average='weighted')

0.8376979863108699

In [20]:
test_df[test_df['real'] == test_df['pred']]

,sent,attn,pred,real
0,다음 에 또 할께요,"[0.7413091063499451, 0.10890647023916245, 0.06...",DIGITAL_P대화종료,DIGITAL_P대화종료
2,정기 결재 \ n 해지 \ n 했으니 \ n \ n 환불 바랍니다,"[0.0003935715649276972, 0.004506119526922703, ...",DIGITAL_P환불문의,DIGITAL_P환불문의
5,이용권 종료 하려구요 ~!,"[0.623607873916626, 0.36070579290390015, 0.014...",DIGITAL_P기타faq문의,DIGITAL_P기타faq문의
6,비밀 쿠폰 이 뭔 가요,"[0.00043983093928545713, 0.9958051443099976, 3...",DIGITAL_P쿠폰요청,DIGITAL_P쿠폰요청
7,삼 쥐 에어컨 찾아줘,"[0.010640528053045273, 0.001728722476400435, 0...",DIGITAL_P추천,DIGITAL_P추천
8,전화 도 안 오고,"[0.49906855821609497, 0.019158028066158295, 0....",DIGITAL_P전화요청,DIGITAL_P전화요청
10,전화 연결 이 되야 물어보죠,"[0.9654471278190613, 0.0026917082723230124, 5....",DIGITAL_P전화요청,DIGITAL_P전화요청
12,항공권 예약 취소 는 어떻게 하나요 ?,"[0.0011822204105556011, 0.006734185386449099, ...",DIGITAL_P주문취소,DIGITAL_P주문취소
14,심심이 아니 ?,"[0.9490893483161926, 0.034988902509212494, 0.0...",DIGITAL_P채팅,DIGITAL_P채팅
15,챗봇 하고 싶은거 다해 !,"[0.6612390875816345, 0.04901933670043945, 0.21...",DIGITAL_P챗봇여부문의,DIGITAL_P챗봇여부문의
